# Visualizando con Altair

Si bien aprendimos a hacer visualizaciones con la librería `matplotlib`, existen varias otras librerías famosas para hacer visualizaciones, entre las que me gustaría destacar `seaborn` y `altair`. Ahora vamos a aprender lo básico de esta última librería, que nos permite hacer visulizaciones a partir de `DataFrames` de `pandas`.

Altair es un _wrapper_ en Python de Vega-Lite. Vega-Lite es "lenguaje de alto nivel para hacer visualizaciones" que está desarrollado en JavaScript. Para ver todas las posibilidades que ofrece Altair, puedes mirar [su documentación oficial](https://altair-viz.github.io/). 

### Visualización de los datos comunales

Vamos a hacer una visualización con `Altair` y los datos comunales. Primero vamos a cargar un `DataFrame` con el nombre, región, presupuesto y población de cada comuna. Recordemos que debemos limpiar los datos "no recepcionado" en presupuesto.

In [55]:
import pandas as pd

df_codigos = pd.read_csv("codigos.csv", delimiter=';')
df_codigos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Código Región       346 non-null    int64 
 1   Nombre Región       346 non-null    object
 2   Abreviatura Región  346 non-null    object
 3   Código Provincia    346 non-null    int64 
 4   Nombre Provincia    346 non-null    object
 5   Código Comuna 2018  346 non-null    int64 
 6   Nombre Comuna       346 non-null    object
dtypes: int64(3), object(4)
memory usage: 19.0+ KB


In [56]:
df_poblacion = pd.read_csv("poblacion.csv", delimiter=';')
df_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CODIGO     345 non-null    int64 
 1   MUNICIPIO  345 non-null    object
 2   POBLACIÓN  345 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.2+ KB


In [57]:
df_presupuesto = pd.read_csv("presupuesto_2019.csv", delimiter=';')
df_presupuesto = df_presupuesto.replace('No Recepcionado', 0)
df_presupuesto['PRESUPUESTO'] = df_presupuesto['PRESUPUESTO'].astype(int)
df_presupuesto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CODIGO       345 non-null    int64 
 1   MUNICIPIO    345 non-null    object
 2   PRESUPUESTO  345 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.2+ KB


Las siguientes líneas nos van a servir para hacer `merge` entre los _datasets_.

In [58]:
df_extended = df_codigos.merge(df_poblacion, left_on='Código Comuna 2018', right_on='CODIGO')
df_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345 entries, 0 to 344
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Código Región       345 non-null    int64 
 1   Nombre Región       345 non-null    object
 2   Abreviatura Región  345 non-null    object
 3   Código Provincia    345 non-null    int64 
 4   Nombre Provincia    345 non-null    object
 5   Código Comuna 2018  345 non-null    int64 
 6   Nombre Comuna       345 non-null    object
 7   CODIGO              345 non-null    int64 
 8   MUNICIPIO           345 non-null    object
 9   POBLACIÓN           345 non-null    int64 
dtypes: int64(5), object(5)
memory usage: 29.6+ KB


In [59]:
df_extended = df_extended.merge(df_presupuesto, left_on='CODIGO', right_on='CODIGO')
df_extended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345 entries, 0 to 344
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Código Región       345 non-null    int64 
 1   Nombre Región       345 non-null    object
 2   Abreviatura Región  345 non-null    object
 3   Código Provincia    345 non-null    int64 
 4   Nombre Provincia    345 non-null    object
 5   Código Comuna 2018  345 non-null    int64 
 6   Nombre Comuna       345 non-null    object
 7   CODIGO              345 non-null    int64 
 8   MUNICIPIO_x         345 non-null    object
 9   POBLACIÓN           345 non-null    int64 
 10  MUNICIPIO_y         345 non-null    object
 11  PRESUPUESTO         345 non-null    int64 
dtypes: int64(6), object(6)
memory usage: 35.0+ KB


Y finalmente nos quedamos con las columnas que necesitamos.

In [60]:
df_extended = df_extended[['Nombre Comuna', 'POBLACIÓN', 'PRESUPUESTO', 'Nombre Región']]
df_extended

,Nombre Comuna,POBLACIÓN,PRESUPUESTO,Nombre Región
0,Iquique,216514,59072234,Tarapacá
1,Alto Hospicio,124150,13984411,Tarapacá
2,Pozo Almonte,16683,7613962,Tarapacá
3,Camiña,1345,1584008,Tarapacá
4,Colchane,1556,2500000,Tarapacá
...,...,...,...,...
340,San Carlos,55935,10459691,Ñuble
341,Coihueco,28147,5001190,Ñuble
342,Ñiquén,11556,3386698,Ñuble
343,San Fabián,4607,1798500,Ñuble


Y ahora vamos a hacer un gráfico interactivo donde se vea el presupuesto en función de la población.

In [70]:
import altair as alt

alt.Chart(df_extended).mark_circle(size=60).encode(
    x='POBLACIÓN',
    y='PRESUPUESTO',
    color='Nombre Región',
    tooltip=['Nombre Comuna', 'Nombre Región']
).interactive()

alt.Chart(...)

Ahora Haremos un gráfico mucho más interactivo.

In [71]:
import altair as alt

selection = alt.selection_multi(fields=['Nombre Región'])
color = alt.condition(selection,
                      alt.Color('Nombre Región:N', legend=None),
                      alt.value('lightgray'))

scatter = alt.Chart(df_extended).mark_circle(size=60).encode(
    x='POBLACIÓN:Q',
    y='PRESUPUESTO:Q',
    color=color,
    tooltip=['Nombre Comuna', 'Nombre Región']
).interactive()


legend = alt.Chart(df_extended).mark_point().encode(
    y=alt.Y('Nombre Región:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

scatter | legend

alt.HConcatChart(...)